# construction d'un reseau de neuronnes

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers

In [5]:
# construction d'un MLP

In [4]:
model = Sequential([
    layers.Input(shape=(28, 28)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

2026-02-18 14:06:27.883222: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# MLP

In [7]:
# MLP plus robuste
model1 = Sequential([
    layers.Input(shape=(28, 28)),

    #bloc 1
    layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    #bloc 2
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(1, activation='sigmoid')
])

# Le CNN robuste

In [8]:
model2 = Sequential([
    layers.Input(shape=(28,28,5)),
    
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.GlobalAveragePooling2D(),

    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Séries Temporelles & Texte (Le RNN / LSTM)

In [14]:
model3 = Sequential([
    layers.Input(shape=(20,20)),
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),

    layers.LSTM(64, return_sequences=False),

    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    
    
])

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential

# --- DONNÉES DE TEST ---
data = """Le chat mange la souris
Le chien dort sur le tapis
Le chat court dans le jardin
Le chien mange sa gamelle"""

# 1. Tokenization (Transformation des mots en nombres)
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1  # +1 pour le padding (0)

# 2. Création de séquences (ex: "Le chat" -> "mange")
input_sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# 3. Padding (S'assurer que toutes les séquences ont la même longueur)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# 4. Séparation X (Entrée) et Y (Cible)
X, y = input_sequences[:,:-1], input_sequences[:,-1]


In [16]:
model = Sequential([
    # Input_dim = taille du vocabulaire, output_dim = taille du vecteur de sens (32)
    layers.Embedding(total_words, 32, input_length=max_sequence_len-1),
    
    # Couche LSTM (64 unités). On ne met pas Flatten car return_sequences=False par défaut
    layers.LSTM(64),
    
    # Régularisation
    layers.Dropout(0.2),
    
    # Sortie : Un neurone par mot dans le dictionnaire
    layers.Dense(total_words, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.summary()


/home/yves/anaconda3/envs/tensflow/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Entraînement
model.fit(X, y, epochs=100, verbose=0) # 100 époques pour cet exemple minuscule

def predire_mot_suivant(texte_depart):
    # Transformer le texte en nombres
    sequence = tokenizer.texts_to_sequences([texte_depart])[0]
    sequence = pad_sequences([sequence], maxlen=max_sequence_len-1, padding='pre')
    
    # Prédire les probabilités
    prediction = model.predict(sequence, verbose=0)
    id_mot_predit = np.argmax(prediction)
    
    # Retrouver le mot correspondant à l'ID
    for mot, index in tokenizer.word_index.items():
        if index == id_mot_predit:
            return mot
    return ""

# --- TEST ---
print(f"Phrase: 'Le chat' -> Suivant: {predire_mot_suivant('mange')}")
print(f"Phrase: 'Le chien' -> Suivant: {predire_mot_suivant('Le chien')}")


Phrase: 'Le chat' -> Suivant: chien
Phrase: 'Le chien' -> Suivant: dort


In [25]:
def generer_paragraphe(seed_text, nb_mots, model, tokenizer, max_sequence_len):
    resultat = seed_text
    
    for _ in range(nb_mots):
        # 1. Préparation de la séquence actuelle
        token_list = tokenizer.texts_to_sequences([resultat])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        
        # 2. Prédiction des probabilités pour le mot suivant
        probabilities = model.predict(token_list, verbose=0)
        
        # 3. Récupération de l'index du mot le plus probable
        predicted_index = np.argmax(probabilities, axis=-1)[0]
        
        # 4. Traduction de l'index en mot
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        
        # 5. Ajout du mot au texte généré et répétition
        resultat += " " + output_word
        
    return resultat

# --- TEST DE GÉNÉRATION ---
# On demande au modèle de générer 5 mots après "Le chat"
paragraphe = generer_paragraphe("Le chien", 4, model, tokenizer, max_sequence_len)
print(f"Texte généré : {paragraphe}")


Texte généré : Le chien dort sur le tapis
